In [62]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from geopy.geocoders import Nominatim
from tqdm import tqdm
from geopy.distance import geodesic
import folium
from folium.plugins import MarkerCluster
import math
import datetime
import geopandas as gpd
import urllib.request
import requests
import json
import openmeteo_requests
import requests_cache
from shapely.geometry import Polygon, Point
from retry_requests import retry
from shapely.wkt import loads
import random 
from pathlib import Path

In [63]:
DATA_DIR_BOMEN = Path("src/data_bomen")
DATA_DIR_GEBOUWEN = Path("src/data_gebouwen")

INCIDENT_DATA_PATH = DATA_DIR_BOMEN / 'Stormdata & FireStations geadresseerd.csv'
BOUWJAAR_DATA_PATH = DATA_DIR_GEBOUWEN / 'BOUWJAAR.csv'
ZIPCODE_JSON_PATH = DATA_DIR_BOMEN / "zipcodes_boxes.json"
GRID_SIZE = 200     ## GRID SIZE IN METERS

TREE_DATA_CLEAN_PATH = DATA_DIR_GEBOUWEN / f"building_geo_data_clean_{str(GRID_SIZE)}.csv"
GRID_DATA_PATH = DATA_DIR_GEBOUWEN / f"grid_enriched_{GRID_SIZE}.csv"
INCIDENTS_WEATHER_PATH = DATA_DIR_BOMEN / "incidents_weather.csv"
INCIDENTS_WEATHER_GEO_PATH = DATA_DIR_GEBOUWEN / f"incidents_weather_geo_{GRID_SIZE}.csv"

POSITIVE_SAMPLES_PATH = DATA_DIR_BOMEN / f"positive_samples{GRID_SIZE}.csv"
NEGATIVE_SAMPLES_PATH = DATA_DIR_BOMEN / f"negative_samples_{GRID_SIZE}.csv"

ZIP_KEY = "Zipcode"
ZIP4_KEY = "Zip4"

DATE_WINDOW = 7

AMSTERDAM_BBOX = (52.26618, 4.64663, 52.475115999999994, 5.150491999999999)

In [64]:
BUILDING_COLUMNS = [
    "OBJECTNUMMER",
    "Bouwjaar",       
    "WKT_LNG_LAT",
    "WKT_LAT_LNG",
    "LNG",
    "LAT"
]

SERVICE_AREAS_OUT_OF_SCOPE = [
    "Amstelveen",
    "Aalsmeer",
    "Uithoorn"
]

RF_INCIDENT_COLUMNS = [
    "Incident_ID",
    "Service_Area",
    "grid_id",
    "Date",
    "Hour",
    "temperature_2m",
    "relative_humidity_2m",
    "dew_point_2m",
    "apparent_temperature",
    "precipitation",
    "rain",
    "snowfall",
    "snow_depth",
    "weather_code",
    "pressure_msl",
    "surface_pressure",
    "wind_speed_10m",
    "wind_direction_10m",
    "wind_gusts_10m",
    "soil_temperature_0_to_7cm",
    "soil_temperature_7_to_28cm",
    "soil_temperature_28_to_100cm",
    "soil_temperature_100_to_255cm",
    "soil_moisture_0_to_7cm",
    "soil_moisture_7_to_28cm",
    "soil_moisture_28_to_100cm",
    "soil_moisture_100_to_255cm",
]

RF_GRID_COLUMNS = [
    "grid_id",
    "has_building",
    "avg_construction_year",
]

RF_BUILDING_COLUMNS = [
    "OBJECTNUMMER",
    "Bouwjaar",       
    "WKT_LNG_LAT",
    "WKT_LAT_LNG",
    "LNG",
    "LAT"
]

In [65]:
def create_grid_gdf():
    geolocator = Nominatim(user_agent="my_geocoder")

    # Get coordinates for Amsterdam
    location = geolocator.geocode("Amsterdam, Netherlands")
    amsterdam_lat, amsterdam_lon = location.latitude, location.longitude

    amsterdam_bbox = AMSTERDAM_BBOX

    # Calculate grid bounds
    lat_step = GRID_SIZE / 111000  # 1 degree of latitude is approximately 111 kilometers
    lon_step = (GRID_SIZE / 111000) / np.cos(np.radians(amsterdam_lat))  # Correct for latitude

    grid_polygons = []
    for lat in np.arange(amsterdam_bbox[0], amsterdam_bbox[2], lat_step):
        for lon in np.arange(amsterdam_bbox[1], amsterdam_bbox[3], lon_step):
            polygon = Polygon([
                (lon, lat),
                (lon + lon_step, lat),
                (lon + lon_step, lat + lat_step),
                (lon, lat + lat_step),
                (lon, lat),
            ])
            grid_polygons.append(polygon)

    grid_gdf = gpd.GeoDataFrame(geometry=grid_polygons, crs="EPSG:4326")
    return grid_gdf

In [66]:
def create_building_incident_gdf(incidents_weather_df, buildings, grid_gdf):
    # Filter on areas in scope
    incidents_weather_df = incidents_weather_df[~incidents_weather_df.Service_Area.isin(SERVICE_AREAS_OUT_OF_SCOPE)]

    # create gdf from buildings
    building_gdf = gpd.GeoDataFrame(buildings, geometry=gpd.points_from_xy(buildings['LNG'], buildings['LAT']), crs="EPSG:4326")
    # create gdf from indicents
    incident_gdf = gpd.GeoDataFrame(incidents_weather_df, geometry=gpd.points_from_xy(incidents_weather_df['LON'], incidents_weather_df['LAT']), crs="EPSG:4326")
    #join with grid gdf
    building_gdf = gpd.sjoin(building_gdf, grid_gdf, how="left", op="within")
    incident_gdf = gpd.sjoin(incident_gdf, grid_gdf, how="left", op="within")

    #clean up gdf
    building_gdf = building_gdf.rename(columns={"index_right" : "grid_id", "geometry" : "location"})
    incident_gdf = incident_gdf.rename(columns={"index_right" : "grid_id", "geometry" : "location"})

    return building_gdf, incident_gdf

In [67]:
def convert_cat_to_avg(
    cat_values,
    delimeter = "-"
):
    ''' 
    Converts categorical values to means of type float
    Splits cat values on delimter, computes the mean for each cat
    Returns mean of all means of the categories
    '''
    means = []
    for cat in cat_values:
        if not isinstance(cat, str):
            continue
        if not delimeter in cat:
            continue
        vals = cat.split(delimeter)
        means.append(np.mean([float(val) for val in vals]))
    m = round(np.mean(means), 3)
    return 0 if np.isnan(m) else m

In [68]:
def enrich_grid_df(
    grid_gdf,
    building_gdf
):
    for i in grid_gdf.index:
        building_sub_df = building_gdf[building_gdf.grid_id == i]
        if len(building_sub_df)>0:
            # Compute and add averages for height, diameter and year
            grid_gdf.at[i, "Bouwjaar"] = round(np.mean(building_sub_df.jaarVanAanleg.values), 3)
            # Add soortnaam counts
            for name, count in building_sub_df.soortnaamKort.value_counts().items():
                grid_gdf.at[i, "has_building"] = True
                grid_gdf.at[i, name] = count
        else:
            grid_gdf.at[i, "has_building"] = False

    return grid_gdf, building_gdf

In [69]:
def save_data(building_gdf, incident_gdf, grid_gdf):
    # save data
    building_gdf.to_csv(TREE_DATA_CLEAN_PATH, sep=",", encoding="utf-8", index=False)
    incident_gdf.to_csv(INCIDENTS_WEATHER_GEO_PATH, sep=",", encoding="utf-8", index=False)

    # clean and save data
    grid_gdf = grid_gdf.fillna(0)
    grid_gdf[grid_gdf.has_building == True]
    grid_gdf['grid_id'] = grid_gdf.index
    grid_gdf.to_csv(GRID_DATA_PATH, sep=",", encoding="utf-8", index=False)


def create_save_positives(incident_gdf, building_gdf, grid_gdf):
    incident_gdf.Date = pd.to_datetime(incident_gdf.Date)
    # Pick necessary columns
    incident_sub_gdf = incident_gdf[RF_INCIDENT_COLUMNS]

    grid_sub_gdf = grid_gdf[RF_GRID_COLUMNS]

    building_gdf = building_gdf.rename(columns={"id" : "tree_id"})
    building_sub_gdf = building_gdf[RF_BUILDING_COLUMNS]
    positive_samples = grid_sub_gdf.merge(incident_sub_gdf, on='grid_id', how='inner')
    positive_samples.to_csv(POSITIVE_SAMPLES_PATH, sep=",", encoding="utf-8", index=False)
    return positive_samples

In [70]:
def verify_sample(
    incidents,
    grid_id,
    date,
    window = DATE_WINDOW
):
    start_date = date - pd.DateOffset(days=window)
    end_date = date + pd.DateOffset(days=window)

    grids = incidents[(incidents['Date'] >= start_date) & (incidents['Date'] <= end_date)].values

    return False if grid_id not in grids else True


def create_save_negatives(
    positives,
    incidents,
    grid
):
    grids_with_building = list(grid[grid.has_building == True].grid_id.values)
    negatives = positives[['Date', 'Hour']]
    negatives[RF_GRID_COLUMNS] = None

    for i, row in negatives.iterrows():
        random_grid = random.sample(grids_with_building, 1)[0]
        while(verify_sample(incidents, random_grid, row.Date)):
            random_grid = random.sample(grids_with_building, 1)[0]
        grid_data = grid[grid.grid_id == random_grid][RF_GRID_COLUMNS].reset_index(drop=True)
        negatives.loc[i, RF_GRID_COLUMNS] = grid_data.iloc[0]
    # save
    negatives.to_csv(NEGATIVE_SAMPLES_PATH, sep=",", encoding="utf-8", index=False)
    return negatives

In [74]:
from src.GetWeather import GetWeather

incident_df = pd.read_csv(INCIDENT_DATA_PATH, sep=",", encoding="utf-8")
incident_df = incident_df.drop(['Unnamed: 0'], axis=1)
incident_df = incident_df.set_index('Incident_ID')

# create datasets
building_df = pd.read_csv(BOUWJAAR_DATA_PATH, sep=";", encoding="utf-8")
building_df['WKT_LNG_LAT'] = building_df['WKT_LNG_LAT'].apply(loads)
building_df['WKT_LAT_LNG'] = building_df['WKT_LAT_LNG'].apply(loads)

incidents_weather_df = pd.read_csv(INCIDENTS_WEATHER_PATH, sep=",", encoding="utf-8")
grid_gdf = create_grid_gdf()

incidents_weather_df = incidents_weather_df[~incidents_weather_df.Service_Area.isin(SERVICE_AREAS_OUT_OF_SCOPE)]
df_tree_incidents = incident_df[incident_df["Damage_Type"]=="Building"]
building_gdf, incident_gdf = create_building_incident_gdf(incidents_weather_df=incidents_weather_df, buildings=building_df, grid_gdf=grid_gdf)

# get rid of unnecessary columns
building_gdf = building_gdf[BUILDING_COLUMNS]
grid_gdf, building_gdf = enrich_grid_df(grid_gdf=grid_gdf, building_gdf=building_gdf)

# save data
save_data(building_gdf=building_gdf, incident_gdf=incident_gdf, grid_gdf=grid_gdf)

positive_samples = create_save_positives(incident_gdf=incident_gdf, building_gdf=building_gdf, grid_gdf=grid_gdf)
negative_samples = create_save_negatives(positives=positive_samples, incidents=incident_gdf, grid=grid_gdf)

weather_getter = GetWeather(grid_path=GRID_DATA_PATH, samples_path=NEGATIVE_SAMPLES_PATH, sleep_time=90)  
negative_samples = weather_getter.add_weather_data()


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Amsterdam%2C+Netherlands&format=json&limit=1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)')))

In [51]:
building_df

,OBJECTNUMMER,Bouwjaar,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT
0,1,2017,"POLYGON ((4.743206 52.404269, 4.742871 52.4032...","POLYGON ((52.404269 4.743206, 52.403299 4.7428...",4.741721,52.403954
1,2,1931,"POLYGON ((4.760418 52.377528, 4.758715 52.3762...","POLYGON ((52.377528 4.760418, 52.37629 4.75871...",4.759217,52.377183
2,3,2017,"POLYGON ((4.798697 52.392101, 4.798704 52.3915...","POLYGON ((52.392101 4.798697, 52.391502 4.7987...",4.798115,52.391720
3,4,2010,"POLYGON ((4.948353 52.354725, 4.947985 52.3543...","POLYGON ((52.354725 4.948353, 52.354396 4.9479...",4.946738,52.355162
4,5,2007,"POLYGON ((4.78266 52.36632, 4.780337 52.364659...","POLYGON ((52.36632 4.78266, 52.364659 4.780337...",4.781296,52.365636
...,...,...,...,...,...,...
22067,22069,1975,"POLYGON ((4.918834 52.371799, 4.918085 52.3713...","POLYGON ((52.371799 4.918834, 52.37133 4.91808...",4.918400,52.371598
22068,22070,1900,"POLYGON ((4.921494 52.371653, 4.921405 52.3715...","POLYGON ((52.371653 4.921494, 52.371598 4.9214...",4.921096,52.371837
22069,22071,2018,"POLYGON ((4.919407 52.345906, 4.918619 52.3457...","POLYGON ((52.345906 4.919407, 52.345738 4.9186...",4.918757,52.346076
22070,22072,2018,"POLYGON ((4.919407 52.345906, 4.919446 52.3459...","POLYGON ((52.345906 4.919407, 52.345915 4.9194...",4.919073,52.345671
